In [ ]:
from time import sleep as sleep 
from os.path import join
import pyautogui as pag
import pyperclip as clip
import os
import shutil
import requests
import asyncio
import aiohttp
import aiofiles
import json

In [ ]:
# ===================================== CONFIGS ==========================================

# If there is a variable path, put a "???" to be replaced by the value in "VARIABLE_PATH"
BASE_URL = "google.com"

# Variable Path
VARIABLE_PATH = [
    ""
]

PATH = join("..", "<path>")

# Ignores a variable that already has a subdirectory in the PATH
IGNORE_DUPLICATES = True

# ========================================================================================

len(VARIABLE_PATH)

In [ ]:
def script(path):
    script = ""
    with open(path, "r", encoding="utf-8") as f:
        script = f.read()

    return script

def assert_path(path) -> bool:
    is_path = os.path.isdir(path)
    if not is_path:
        os.mkdir(path)
    
    return is_path

def fetch(path, image_url):
    try:
        print(image_url)
        r = requests.get(image_url, stream = True)
        r.raw.decode_content = True

        name = image_url.split("/")[-1]

        with open(join(path, name),'wb') as f:
            shutil.copyfileobj(r.raw, f)
    except:
        fetch(path, image_url)

def fetch_all(path, data):
    assert_path(path)

    for image_url in data:
        fetch(path, image_url)
        

# Fetch a single Image 
# https://stackoverflow.com/questions/35388332/how-to-download-images-with-aiohttp
async def async_fetch(session: aiohttp.ClientSession, url: str, path: str) -> str:
    async with session.get(url) as resp:
        if resp.status == 200:
            name = url.split("/")[-1]
            async with aiofiles.open(join(path, name), mode='wb') as f:
                await f.write(await resp.read())
            return ""
        else:
            return url

# Fetch all URLs
async def async_fetch_all(session: aiohttp.ClientSession, urls: str, path: str) -> list[str]:
    assert_path(path)
    tasks = []
    for url in urls:
        task = asyncio.create_task(async_fetch(session, url, path))
        tasks.append(task)
    
    # Return the unsuccessfull requests
    res = await asyncio.gather(*tasks)
    res = [url for url in res if url != ""]
    return res 
        

In [ ]:
var_dir = os.listdir(PATH)
length = len(VARIABLE_PATH)
done = 1
pag.PAUSE = 0.3

len(os.listdir("../kill_me_when_seen/"))
JSON_PATH = join(PATH, "jsons")
assert_path(JSON_PATH)

# Gives 3 seconds to focus on a browser window (use chrome, to be sure)
sleep(3)

for urls in VARIABLE_PATH: # VARIABLE_PATH:
    print(f"{urls} => {done} / {length}")

    if urls in var_dir or f"{urls}.json" in os.listdir(JSON_PATH):
        print("SKIPPED - Already has a subdirectory or json")
        done += 1
        continue 

    # Focus on the search bar and puts the URL
    pag.hotkey('ctrl', 'l', interval=0.3)
    clip.copy(BASE_URL.replace("???", str(urls)))
    pag.hotkey("ctrl", "v", interval=0.3)
    pag.press("enter")

    # Awaits for the site to load
    sleep(5)

    # Opens terminal
    pag.hotkey('ctrl', 'shift', 'j')

    # Focus on console
    pag.hotkey("ctrl", "'", interval=0.5)

    # Puts the script
    clip.copy(script("../clover.js"))
    pag.hotkey("ctrl", "v", interval=0.5)
    pag.press("enter")

    # Focus on the window, for the clipboard script to work
    pag.hotkey("alt", "tab", interval=0.5)
    sleep(6)
    
    # Pastes the returning output
    data = clip.paste()
    data = data.split(",")
    
    with open(join(JSON_PATH, f"{urls}.json"), "w") as f:
        json.dump(data, f)
        
    # Sleep, because the program needs a rest too...
    sleep(3)
    done += 1

# Rick rolls, for some reason
pag.hotkey('ctrl', 'l', interval=0.3)
clip.copy("https://www.youtube.com/watch?v=dQw4w9WgXcQ")
pag.hotkey("ctrl", "v", interval=0.3)
pag.press("enter")


In [ ]:
done = 1
length = len(VARIABLE_PATH)
for url in VARIABLE_PATH:
    print(f"{url} => {done} / {length}")
    # Read the json, if it doesn't exists, continue the loop
    try:
        with open(join(JSON_PATH, f"{url}.json"), "r") as f:
            leftovers = json.load(f)
    except FileNotFoundError:
        done += 1
        continue

    if not leftovers:
        done += 1
        continue 

    # Looping until all the urls have been taken care of
    while leftovers:
        # Do whatever with the scraped data
        try:
            async with aiohttp.ClientSession() as session:
                leftovers = await async_fetch_all(session, leftovers, join(PATH, url))
                print(leftovers)
                # # Handling unsuccessfull requests in an async environment is tough, so just let the sync version do that  
                # if leftovers:
                #     fetch_all(join(PATH, urls), leftovers)
        except aiohttp.InvalidURL: 
            break
        except TimeoutError:
            print("Oopsie :()")
    # Remove the json
    # os.remove(join(PATH, url, f"{url}.json"))
    done += 1